This notebook does some data exploration.

Main findings:
- 54% of the movies make more revenue in domestic than in international
- Most of the movies are between 95 and 120 minutes.
- There is a positive correlation between budget and revenue
- The total revenue of a movie is in average 3.25x the budget

### Import libraries

In [ ]:
import pandas as pd

### Data importation

In [ ]:
data = pd.read_csv("/kaggle/input/movie-attributes-for-3400-movies-from-20002020/Attributes_DataFrame.csv")
daily_df = pd.read_csv("/kaggle/input/movie-attributes-for-3400-movies-from-20002020/Daily_DataFrame.csv")

# Data visualization / exploration

In [ ]:
data.shape

In [ ]:
data.head()

Creating year of the movie feature

In [ ]:
data["year"] = data.Title.apply(lambda x: x.split("(")[-1].split(")")[0])
data['revenue_billion'] = (data.Domestic + data.International) / 10**6
data['budget_billion'] = data.Budget / 10**6
data = data.dropna(subset=['revenue_billion', 'budget_billion'])

In [ ]:
data

In [ ]:
data.head()

See if there is movie with more money made in domestic country than in International

In [ ]:
sum([d > i for d, i in zip(data.Domestic, data.International)]) / data.shape[0]

More than half of the movies made more money in domestic than in International

In [ ]:
data[data.Domestic > data.International]

Distribution of the runtime in minutes

In [ ]:
data.hist('Runtime', bins=5)

Most of the movies are between 95 and 120 minutes.

In [ ]:
data.plot.scatter(x='revenue_billion', y='Budget')

**It seems there is a positive correlation between Bugdet and revenue in billion. /!\ Correlation doesn't imply causality.**

In [ ]:
data.describe()

In [ ]:
daily_df.shape

In [ ]:
daily_df.head()

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
lr = LinearRegression()
lr.fit(X=data.budget_billion.values.reshape(-1, 1), y=data.revenue_billion.values)

In [ ]:
lr.coef_

In [ ]:
import statsmodels.api as sm

X=data.budget_billion.values.reshape(-1, 1)
y=data.revenue_billion.values

X2 = sm.add_constant(X)
est = sm.OLS(y, X2)
est2 = est.fit()
print(est2.summary())

**The p-value associated to the variable budget is lower than 1%, we can say that the explanatory variable is significant at 1%. In average, the revenue is 3.25 times the budget of the film.**

# Daily revenue prediction

In [ ]:
daily_df[daily_df["Movie_Title"] == "Jurassic World (2015)"]

### Daily revenue is clearly decreasing over time

In [ ]:
for movie in daily_df.Movie_Title.unique()[:3]:
    print(daily_df[daily_df["Movie_Title"] == movie].plot.scatter(x="Date", y="Daily", title=movie))

In [ ]:
!pip install autoforecast

In [ ]:
from autoforecast.models.time_series import ARMA